# Grid Search Cross-Validation

Using the `esi_hparams_search` function you can search for the appropriate parameters for both `esi_griddata` and `esi_nongriddata`.

The method used is `k-fold`. The `esi_hparams_search` function receives the integer parameter `k`, which is the cut-off for `k-fold`.  If `k` is equal to `-1` or equal to the number of points, `leave-one-out` is used. The rest of the parameters are identical to those received by `esi_griddata` and `esi_nongriddata` except that they must be lists containing the possible options for the search.

In [2]:
from spatialize.gs.esi import esi_hparams_search
import spatialize.gs.esi.aggfunction as af

## Synthetic data generation

This example is taken from the documentation of the scipy griddata function ([here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html)).

In [3]:
import numpy as np
def func(x, y):  # a kind of "cubic" function
    return x * (1 - x) * np.cos(4 * np.pi * x) * np.sin(4 * np.pi * y ** 2) ** 2

grid_x, grid_y = np.mgrid[0:1:100j, 0:1:200j]

rng = np.random.default_rng()
points = rng.random((1000, 2))
values = func(points[:,0], points[:,1])

## Grid search cross-validation for kriging as the base interpolator

In [4]:
b_params = esi_hparams_search(points, values, (grid_x, grid_y),
                              local_interpolator="kriging", griddata=True, k=10,
                              model=["spherical", "exponential", "cubic", "gaussian"],
                              nugget=[0.0, 0.5, 1.0],
                              range=[10.0, 50.0, 100.0, 200.0],
                              alpha=list(np.flip(np.arange(0.95, 0.97, 0.01))))
b_params

## Grid search cross-validation for IDW as the base interpolator


### Starting with a broad search

In [5]:
b_params = esi_hparams_search(points, values, (grid_x, grid_y),
                              local_interpolator="idw", griddata=True, k=10,
                              exponent=list(np.arange(1.0, 15.0, 1.0)),
                              alpha=(0.5, 0.6, 0.8, 0.9, 0.95))
b_params

### Now refining the search for `alpha` and the aggregation function

In [4]:
b_params = esi_hparams_search(points, values, (grid_x, grid_y),
                              local_interpolator="idw", griddata=True, k=10,
                              exponent=list(np.arange(1.0, 15.0, 1.0)),
                              alpha=(0.95, 0.96, 0.97, 0.975, 0.98),
                              agg_function={"mean": af.mean,
                                            "median": af.median,
                                            "p25": af.Percentile(25),
                                            "p75": af.Percentile(75)
                                            })
b_params